In [238]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


Задание ПРО (новое)

1. Повторите все те же шаги, что и в базовой-версии, но добавьте еще несколько валют для исследования. Для каждой пары 
валют создайте отдельную таблицу.

2. Изменения также коснутся и итоговой таблицы, в который для каждой новой исследуемой пары валют вам необходимо 
произвести новые расчеты, но на этом шаге они все должны быть в одной таблице.

3. Соберите всю логику в отдельное python-приложение и добавьте в docker-compose совместно с сервисом для СУБД Postgres. 
Учитывайте особенности установки дополнительных модулей языка Python.


В качестве результата предоставьте docker-compose файл, который содержит 2 сервиса:

    - СУБД Postgres,

    - Python-приложение, которое реализует логику из задания.

 

По итогу выполнения всех пунктов задания вам необходимо прислать ссылку на ваш репозиторий.

 
В качестве результата предоставьте python-скрипт, который реализует описанную выше логику.

1. Парсим данные с сайта:

In [10]:
import requests       #для работы с web страницами
import pandas as pd   #для работы с датафреймами
import psycopg2
import json           #для преобразования ответов в структурирвоанный json
import numpy as np    #для математических операций
from datetime import datetime  #для работы с датами
from pathlib import Path

In [11]:
BASE_URL = 'https://api.exchangerate.host/timeseries?'
# URL метода 

In [94]:
start_date = '2023-01-01'
end_date = '2023-01-31'
currency = ['BTC', 'CNY', 'AED'] #добавили еще две валюты юань и дирхам
symbols = 'RUB'
format = 'CSV'

In [13]:
for base in currency:
    response = requests.get(BASE_URL, params={'base': base,
                                      'start_date': start_date,
                                      'end_date': end_date,
                                      'symbols': symbols,
                                      'format': format
                                })
    file = open(f'./files/jan2023_{base}.csv', 'wb')               # создаем отдельные файлы Jan2023_{валюта}.csv
    file.write(response.content)
    file.close()
    

In [112]:
for base in currency:
    df = pd.read_csv(f'./files/jan2023_{base}.csv', delimiter=',', decimal=',', index_col=False)
    df = pd.DataFrame(df)
    df.to_csv(f'./files/jan2023_{base}.csv', decimal=',', index=False)
    print(df.head(2))                                       #убедимся, что все как надо
                                            

         date code          rate base  start_date    end_date
0  2023-01-01  RUB  1.231483e+06  BTC  2023-01-01  2023-01-31
1  2023-01-02  RUB  1.208671e+06  BTC  2023-01-01  2023-01-31
         date code       rate base  start_date    end_date
0  2023-01-01  RUB  10.683470  CNY  2023-01-01  2023-01-31
1  2023-01-02  RUB  10.510794  CNY  2023-01-01  2023-01-31
         date code       rate base  start_date    end_date
0  2023-01-01  RUB  20.065046  AED  2023-01-01  2023-01-31
1  2023-01-02  RUB  19.739792  AED  2023-01-01  2023-01-31


In [113]:
data_dir = Path('./files/')
df1 = pd.concat([pd.read_csv(f) for f in data_dir.glob("*.csv")], ignore_index=True) # соединим все в один файл
df2=df1.drop(['start_date', 'end_date'], axis=1)            # удалим столбцы start и end date
      

In [114]:
df2.to_csv('./files/jan2023_total.csv', sep=',', decimal=',', index=False)
df2['rate']= df2['rate'].str.replace(',', '.').astype(float)
df2.info()            # проверили, что у нас верное кол-во записей

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    93 non-null     object 
 1   code    93 non-null     object 
 2   rate    93 non-null     float64
 3   base    93 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.0+ KB


In [115]:
print(df2.head(5))

         date code       rate base
0  2023-01-01  RUB  20.065046  AED
1  2023-01-02  RUB  19.739792  AED
2  2023-01-03  RUB  19.886329  AED
3  2023-01-04  RUB  19.741493  AED
4  2023-01-05  RUB  19.636388  AED


2.Проведем анализ:
1) день, в который значение курса было максимальным,

2) день, в который значение курса было минимальным,

3) максимальное значение курса,

4) минимальное значение курса,

5) среднее значение курса за весь месяц,

6) значение курса на последний день месяца.

In [116]:
  finalDict = list()
for base in currency:
    df3 = pd.read_csv(f'./files/jan2023_{base}.csv', sep=',', decimal=',', index_col=False)
    df3 = pd.DataFrame(df3) 
    day_max = df3['date'].max(axis=0)         # проведем анализ по валютам: день максимума, день минимума, 
    day_min = df3['date'].min(axis=0)
    min_rate = df3['rate'].min()                       # максимальное, минимальное
    max_rate = df3['rate'].max()
    df3['rate'] = pd.to_numeric(df3['rate'], downcast="float")
    avg_rate=round(df3['rate'].mean(),2)                        # и среднее значение
    data = {'Currency': base,'Day_max rate': day_max,'Max rate': max_rate, 'Day_min rate': day_min, 'Min rate': min_rate,'Avg rate':avg_rate} # создадим датафрейм
    finalDict.append(data)          
    print()
    print(f'День с максимальным значением валюты {base}:', day_max,'Значение курса', max_rate)
    print(f'День с минимальным значением валюты {base}:', day_min,'Значение курса', min_rate)
    print(f'Среднее значение валюты {base}:', avg_rate)
    print()
    df4=pd.DataFrame.from_dict(finalDict, orient='columns')
    df4.to_csv('./files/jan2023_analysis.csv', decimal=',', index=False)       # записали в отдельный файлик все рассчитанные значения
print(df4) 


День с максимальным значением валюты BTC: 2023-01-31 Значение курса 1670687.543478
День с минимальным значением валюты BTC: 2023-01-01 Значение курса 1198845.096774
Среднее значение валюты BTC: 1413587.7


День с максимальным значением валюты CNY: 2023-01-31 Значение курса 10.68347
День с минимальным значением валюты CNY: 2023-01-01 Значение курса 9.845475
Среднее значение валюты CNY: 10.31


День с максимальным значением валюты AED: 2023-01-31 Значение курса 20.065046
День с минимальным значением валюты AED: 2023-01-01 Значение курса 17.96626
Среднее значение валюты AED: 19.06

  Currency Day_max rate      Max rate Day_min rate      Min rate      Avg rate
0      BTC   2023-01-31  1.670688e+06   2023-01-01  1.198845e+06  1.413588e+06
1      CNY   2023-01-31  1.068347e+01   2023-01-01  9.845475e+00  1.031000e+01
2      AED   2023-01-31  2.006505e+01   2023-01-01  1.796626e+01  1.906000e+01


3. Нужно записать полученные данные в СУБД Постгрес:

In [122]:
db_con = psycopg2.connect(database='exrate',     # создадим подключение к созданной базе данных exrate
                        user='postgres',
                        password='pass',
                        host='localhost',
                        port=5435)
cur = db_con.cursor()

In [123]:
cur.execute(""" CREATE TABLE rates_jan2023(
    date DATE,
    val_id VARCHAR,
    base_rate DECIMAL,
    val_base VARCHAR
)
""") 
db_con.commit()         # создадим общую таблицу rates_jan2023  

In [124]:
cur.execute(""" COPY rates_jan2023 
FROM '/tmp/jan2023_total.csv'
DELIMITER ';'
CSV HEADER;
""")
db_con.commit() # запишем файл с курсом в таблицу  (предварительно передали файл в контейнер командой docker cp 


In [284]:
cur.execute("""select * from rates_jan2023;""")            #проверка записи
print(cur.fetchone())
db_con.commit()

(0, datetime.date(2023, 1, 1), 'RUB', Decimal('1231482.765625'), 'BTC')


In [118]:
cur.execute("""                                     
CREATE TABLE rates_analysis(
    Currency VARCHAR,
    Date_max DATE ,
    Max_rate DECIMAL,
    Date_min DATE ,
    Min_rate DECIMAL,
    Avg_rate DECIMAL
    
)
""")  
db_con.commit()    # создадим таблицу c анализом курса  

In [121]:
cur.execute(""" COPY rates_analysis 
FROM '/tmp/jan2023_analysis.csv'
DELIMITER ';'
CSV HEADER;""")
db_con.commit() # запишем файл с курсом в таблицу  (предварительно передали файл в контейнер командой docker cp C:\Users\jsb24\.docker\exdb\analysis_BTC_Jan2023.csv ex_cont:/tmp/)

In [291]:
cur.execute("""select * from btc_analysis;""")            #проверка записи
print(cur.fetchall())
db_con.commit()

[(0, datetime.date(2023, 1, 31), Decimal('1670687.543478'), datetime.date(2023, 1, 1), Decimal('1198845.096774'), Decimal('1413587.697503516'))]
